In [1]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
import datetime
import os
import numpy as np
import tensorflow as tf

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix

In [3]:
# dataset = "CHASEDB1"
# dim = 192

# pred_file = "preds_EffnetB7_R2_2021-02-19 07:11:45.npz"

# main_path = f"/content/gdrive/My Drive"

In [4]:
dataset = "DRIVE"
dim = 96

pred_file = "preds_EffnetB7_R2_2021-02-19 18:07:57.npz"

main_path = f"/content/gdrive/My Drive"

In [5]:
testing_labels = f"{main_path}/Data/{dataset}/patches_{dim}/labels/test/"

test_lbl = next(os.walk(testing_labels))[2]
test_lbl.sort()

Y_test = np.concatenate([np.load(testing_labels + file_id)["arr_0"] for file_id in test_lbl], axis=0)

if dataset == "CHASEDB1":
    Y_test = Y_test.astype("float64")
else:
    Y_test = Y_test / 255

In [6]:
print(np.max(Y_test))
print(np.min(Y_test))

1.0
0.0


In [7]:
# offset = 8
# target = Y_test.shape[1] - offset * 2
# print(offset, target)

In [8]:
Y_test.shape

(1000, 96, 96)

In [9]:
Y_test = np.reshape(Y_test, (Y_test.shape[0], Y_test.shape[1], Y_test.shape[2], 1))
# Y_test = tf.image.crop_to_bounding_box(Y_test, offset, offset, target, target).numpy()

In [10]:
Y_test.shape

(1000, 96, 96, 1)

In [11]:
Y_pred = np.load(f"{main_path}/Data/{dataset}/patches_{dim}/predictions/{pred_file}")["arr_0"]

KeyboardInterrupt: ignored

In [ ]:
Y_pred.shape

In [ ]:
# Y_pred = tf.image.crop_to_bounding_box(Y_pred, offset, offset, target, target).numpy()

In [ ]:
Y_pred.shape

In [ ]:
def dice_coef(y_true, y_pred):
    smooth = 0
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

In [ ]:
dice = dice_coef(Y_test, Y_pred)

In [ ]:
Y_test = Y_test.flatten()
Y_pred = Y_pred.flatten()

In [ ]:
def image_metrics(Y_test, Y_pred, lim=0.5):
    Y_pred_bin = np.zeros_like(Y_test)
    idx = Y_pred > lim
    Y_pred_bin[idx] = 1

    auc = roc_auc_score(Y_test, Y_pred)
    f1 = f1_score(Y_test, Y_pred_bin)
    acc = accuracy_score(Y_test, Y_pred_bin)
    mat = confusion_matrix(Y_test, Y_pred_bin)

    TN = mat[0][0]
    FN = mat[1][0]
    TP = mat[1][1]
    FP = mat[0][1]

    sen = TP/(TP+FN)
    spe = TN/(TN+FP)

    return auc, f1, acc, sen, spe

In [ ]:
start = datetime.datetime.now()

In [ ]:
auc, f1, acc, sen, spe = image_metrics(Y_test, Y_pred, lim=0.5)

In [ ]:
print(f"Accuracy = \t{acc}")
print(f"f1score\t = \t{f1}")
print(f"AUC\t = \t{auc}")
print(f"Dice\t = \t{dice}")
print(f"Sensitivity = \t{sen}")
print(f"Specificity = \t{spe}")

In [ ]:
end = datetime.datetime.now()

In [ ]:
diff = end - start

In [ ]:
start

In [ ]:
diff

In [ ]:
with open(f"{main_path}/Data/{dataset}/results_{str(datetime.datetime.now())[:-7]}.txt", "w") as ff:
    ff.write(f"Accuracy = \t{acc}\n")
    ff.write(f"f1score\t = \t{f1}\n")
    ff.write(f"AUC\t = \t{auc}\n")
    ff.write(f"Dice\t = \t{dice}\n")
    ff.write(f"Sensitivity = \t{sen}\n")
    ff.write(f"Specificity = \t{spe}\n")

In [ ]:
print("----------Fin----------")